# Test cases for shutil

## Convert index between SH coefficient array and vector

When solving boundary value problems, the SH coefficient array is flattened into vector. The vectors become matrix so that the linear combination is performed by matrix-vector multiplication.

In [1]:
import sys, os
sys.path.append('..')
modedir = '../shelastic/default_modes'
import pyshtools
import numpy as np
import scipy.sparse as spm
from shelastic.shutil import lm2L, L2lm, LM_list, ILM_list, l_coeffs, SHCilmToVector, SHVectorToCilm

In [2]:
lmax = 3
# generate a coefficent structure with l degrees
print('l_coeffs(%d):'%lmax)
print(l_coeffs(lmax))

l_coeffs(3):
[[[0 0 0 0]
  [1 1 1 1]
  [2 2 2 2]
  [3 3 3 3]]

 [[0 0 0 0]
  [1 1 1 1]
  [2 2 2 2]
  [3 3 3 3]]]


In [3]:
# generate a 1d list of l,m numbers following the order
print('LM_list(%d):'%lmax)
print(LM_list(lmax))

LM_list(3):
(array([0, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3]), array([ 0, -1,  0,  1, -2, -1,  0,  1,  2, -3, -2, -1,  0,  1,  2,  3]))


In [4]:
# generate a 1d list of i,l,m index following the order
print('ILM_list(%d):'%lmax)
print(ILM_list(lmax))

ILM_list(3):
(array([0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0]), array([0, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3]), array([0, 1, 0, 1, 2, 1, 0, 1, 2, 3, 2, 1, 0, 1, 2, 3]))


In [5]:
# convert between a coefficient array to 1d vector
A = pyshtools.SHCoeffs.from_random(power=np.arange(lmax+1)).to_array()
print('Random SHcoeffs:')
print(A)
vecA = SHCilmToVector(A, lmax=5)
print('Convert to 1d vector representation:')
print(vecA)
print('Convert back to coefficient array:')
newA = SHVectorToCilm(vecA, lmax=lmax)
print(newA)

Random SHcoeffs:
[[[-0.          0.          0.          0.        ]
  [ 0.71916532 -0.38470517  0.          0.        ]
  [-0.03165646 -0.65132493 -0.20555647  0.        ]
  [-0.42896651 -1.01498513  0.04741038  0.34362494]]

 [[ 0.          0.          0.          0.        ]
  [ 0.          0.25011399  0.          0.        ]
  [ 0.          0.29902711 -0.23413305  0.        ]
  [ 0.          0.05326745  0.61720109  0.43657629]]]
Convert to 1d vector representation:
[-0.          0.25011399  0.71916532 -0.38470517 -0.23413305  0.29902711
 -0.03165646 -0.65132493 -0.20555647  0.43657629  0.61720109  0.05326745
 -0.42896651 -1.01498513  0.04741038  0.34362494  0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]
Convert back to coefficient array:
[[[-0.          0.          0.          0.        ]
  [ 0.71916532 -0.384705

In [6]:
test_convert = np.allclose(A, newA)
if test_convert:
    print('Conversion between Cilm and Vector consistent with each other')
else:
    print('Conversion between Cilm and Vector NOT consistent')

Conversion between Cilm and Vector consistent with each other


## Spherical harmonic fundamental modes

Test whether the generated modes are consistent with our generated mode.

In [7]:
from shelastic.shelastic import generate_modes
Umodes, Smodes, Tmodes = generate_modes(5, save_lmax=8, etol=1e-14)

0 0 0
irregular solid harmonic modes...
regular solid harmonic modes...
0 0 1
irregular solid harmonic modes...
regular solid harmonic modes...
0 0 2
irregular solid harmonic modes...
regular solid harmonic modes...
1 -1 0
irregular solid harmonic modes...
regular solid harmonic modes...
1 -1 1
irregular solid harmonic modes...
regular solid harmonic modes...
1 -1 2
irregular solid harmonic modes...
regular solid harmonic modes...
1 0 0
irregular solid harmonic modes...
regular solid harmonic modes...
1 0 1
irregular solid harmonic modes...
regular solid harmonic modes...
1 0 2
irregular solid harmonic modes...
regular solid harmonic modes...
1 1 0
irregular solid harmonic modes...
regular solid harmonic modes...
1 1 1
irregular solid harmonic modes...
regular solid harmonic modes...
1 1 2
irregular solid harmonic modes...
regular solid harmonic modes...
2 -2 0
irregular solid harmonic modes...
regular solid harmonic modes...
2 -2 1
irregular solid harmonic modes...
regular solid harmo

In [8]:
from scipy.io import loadmat, savemat

Umodes_def = loadmat(os.path.join(modedir, 'Umodes.mat'))
Smodes_def = loadmat(os.path.join(modedir, 'Smodes.mat'))
Tmodes_def = loadmat(os.path.join(modedir, 'Tmodes.mat'))

In [9]:
from shelastic.shbv import generate_submat
from scipy.sparse.linalg import norm

lmax_row = 8; lmax_col = 5; shtype = 'reg';
mu = 2.65; nu = 0.347;

Dmat_def = generate_submat(Umodes_def, mu, nu, lmax_col, lmax_row, shtype=shtype, verbose=True)
Dmat = generate_submat(Umodes, mu, nu, lmax_col, lmax_row, shtype=shtype, verbose=True)

diffDmat = Dmat - Dmat_def
test_generate_modes = norm(diffDmat) < 1e-12
if test_generate_modes:
    print('generate_modes test passed')
else:
    print('generate_modes test failed')

Integrating modes to a matrix
243 108
Integrating modes to a matrix
243 108
generate_modes test passed


## Test results

In [10]:
if test_convert and test_generate_modes:
    print('passed all tests')

passed all tests
